In [1]:
import os
import pandas as pd
import numpy as np
import editdistance
import time 


from PIL import Image
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import timm  

from sklearn.model_selection import train_test_split

from transformers import TrOCRProcessor, VisionEncoderDecoderModel

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_dir = os.getcwd()

# Same paths as your original code
ground_truth_path = os.path.join(base_dir, 'balinese_transliteration_train.txt') 
images_dir        = os.path.join(base_dir, 'balinese_word_train')

filenames = []
labels    = []

with open(ground_truth_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:  # Ensure the line is not empty
            parts = line.split(';')
            if len(parts) == 2:
                filename, label = parts
                label = label.lower()
                filenames.append(filename)
                labels.append(label)
            else:
                print(f"Skipping malformed line: {line}")

data = pd.DataFrame({
    'filename': filenames,
    'label': labels
})

label_counts = data['label'].value_counts()

all_text = ''.join(data['label'])
unique_chars = sorted(list(set(all_text)))

# Create character->index starting from 1
char_to_idx = {char: idx + 1 for idx, char in enumerate(unique_chars)}
# Add special tokens
char_to_idx['<PAD>'] = 0
char_to_idx['<UNK>'] = len(char_to_idx)
char_to_idx['<SOS>'] = len(char_to_idx)
char_to_idx['<EOS>'] = len(char_to_idx)

# Reverse mapping
idx_to_char = {v: k for k, v in char_to_idx.items()}

vocab_size = len(char_to_idx)
print(f"Vocabulary size: {vocab_size}")

def encode_label(label, char_to_idx, max_length):
    """
    Converts a label (string) into a list of indices with <SOS>, <EOS>, padding, etc.
    """
    encoded = (
        [char_to_idx['<SOS>']] +
        [char_to_idx.get(ch, char_to_idx['<UNK>']) for ch in label] +
        [char_to_idx['<EOS>']]
    )
    # Pad if needed
    if len(encoded) < max_length:
        encoded += [char_to_idx['<PAD>']] * (max_length - len(encoded))
    else:
        encoded = encoded[:max_length]
    return encoded

max_label_length = max(len(label) for label in data['label']) + 2  # +2 for <SOS> and <EOS>
data['encoded_label'] = data['label'].apply(lambda x: encode_label(x, char_to_idx, max_label_length))
data['label_length']  = data['label'].apply(len)

rare_labels = label_counts[label_counts < 3].index  # NEW: words that appear <3 times

def custom_split(df, rare_label_list, test_size=0.1, random_state=42):
    # Separate rare words from frequent ones
    rare_df     = df[df['label'].isin(rare_label_list)]
    non_rare_df = df[~df['label'].isin(rare_label_list)]

    #  train/val split for non-rare
    train_nr, val_nr = train_test_split(non_rare_df, test_size=test_size, 
                                        random_state=random_state)

    # Combine rare samples entirely into training
    train_df = pd.concat([train_nr, rare_df], ignore_index=True)
    # Shuffle after combining
    train_df = train_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    val_df = val_nr.reset_index(drop=True)
    return train_df, val_df

# Call custom_split instead of direct train_test_split
train_data, val_data = custom_split(data, rare_labels, test_size=0.1, random_state=42) 

print(f"Training size: {len(train_data)}; Validation size: {len(val_data)}")

Vocabulary size: 39
Training size: 13972; Validation size: 1050


In [3]:
class BalineseDataset(Dataset):
    def __init__(self, df, images_dir, transform=None):
        self.data       = df.reset_index(drop=True)
        self.images_dir = images_dir
        self.transform  = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name     = self.data.loc[idx, 'filename']
        label        = self.data.loc[idx, 'encoded_label']
        label_length = self.data.loc[idx, 'label_length']

        img_path = os.path.join(self.images_dir, img_name)
        image    = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.long)
        return image, label, torch.tensor(label_length, dtype=torch.long)

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

train_dataset = BalineseDataset(train_data, images_dir, transform=transform)
val_dataset   = BalineseDataset(val_data,   images_dir, transform=transform)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [5]:
class ResNet18Encoder(nn.Module):
    """
    Encoder that uses a pretrained ResNet18 to extract features of shape 
    [B, H*W, C], which the DecoderRNN can then attend over.
    """

    def __init__(self, pretrained=True):
        super(ResNet18Encoder, self).__init__()
        resnet = models.resnet18(pretrained=pretrained)

        # Remove the classification (fc) layer
        modules = list(resnet.children())[:-2]  # remove the avgpool & fc
        self.cnn = nn.Sequential(*modules)

        # last convolutional block outputs 512 channels
        self.encoder_dim = 512

    def forward(self, x):
        """
        Input shape:  x -> [batch_size, 3, 224, 224]
        Output shape: -> [batch_size, num_patches, encoder_dim]
                       where num_patches = H' * W' from the final feature map
        """
        # pass through ResNet (up to layer4)
        features = self.cnn(x)  # [B, 512, H', W']

        # Flatten the spatial dims
        # shape => [B, 512, H', W'] -> [B, H'*W', 512]
        b, c, h, w = features.shape
        features = features.permute(0, 2, 3, 1)   # [B, H', W', C]
        features = features.reshape(b, -1, c)     # [B, H'*W', C]

        return features

In [6]:
class ResNet50Encoder(nn.Module):
    """
    Encoder that uses a pretrained ResNet50 to extract features of shape 
    [B, H*W, C]. For ResNet50, the final block has 2048 output channels.
    """

    def __init__(self, pretrained=True):
        super(ResNet50Encoder, self).__init__()
        resnet = models.resnet50(pretrained=pretrained)

        # remove the avgpool & fc layers
        modules = list(resnet.children())[:-2]  
        self.cnn = nn.Sequential(*modules)

        # For ResNet50, the last block has 2048 output channels
        self.encoder_dim = 2048

    def forward(self, x):
        # x -> [B, 3, 224, 224]
        features = self.cnn(x)  # [B, 2048, H', W']

        b, c, h, w = features.shape
        features = features.permute(0, 2, 3, 1)   # [B, H', W', C]
        features = features.reshape(b, -1, c)     # [B, H'*W', C]

        return features

In [7]:
class ViTEncoder(nn.Module):
    """
    A simple ViT encoder that extracts patch embeddings as [batch_size, num_patches, hidden_dim].
    We'll use timm to load a pretrained ViT. Then we use .forward_features() to get a
    feature map of shape [B, C, H', W'] for many timm ViT models, which we flatten.
    """
    def __init__(self, model_name="vit_large_patch16_224", pretrained=True):
        super(ViTEncoder, self).__init__()
        self.vit = timm.create_model(model_name, pretrained=pretrained)
        # Remove or replace the classification head
        self.vit.head = nn.Identity()

        # timm's ViT typically has an embed_dim attribute
        self.encoder_dim = self.vit.embed_dim

    def forward(self, x):
        """
        :param x: [batch_size, 3, 224, 224]
        :return:  [batch_size, num_patches, encoder_dim]
        """
        # forward_features usually returns [B, hidden_dim, H', W'] or [B, hidden_dim]
        feats = self.vit.forward_features(x)  # [B, hidden_dim, 14, 14] for vit_base_patch16_224

        # Flatten the spatial dimensions
        if feats.dim() == 4:  # [B, C, H, W]
            b, c, h, w = feats.shape
            feats = feats.permute(0, 2, 3, 1).reshape(b, -1, c)  # => [B, H*W, C]

        return feats

In [8]:
class SwinEncoder(nn.Module):
    def __init__(self, model_name="swin_small_patch4_window7_224", pretrained=True):
        """
        A simple Swin Transformer encoder that extracts patch embeddings
        as [batch_size, num_patches, hidden_dim]. We'll use timm to load 
        a pretrained Swin model, remove its classification head, then flatten.
        """
        super().__init__()
        self.swin = timm.create_model(model_name, pretrained=pretrained)
        self.swin.head = nn.Identity()

        # We'll assign encoder_dim dynamically after forward
        self.encoder_dim = None

    def forward(self, x):
        """
        :param x: [batch_size, 3, 224, 224]
        :return:  [batch_size, num_patches, encoder_dim]
        """
        feats = self.swin.forward_features(x)            # [B, C, H, W]
        b, c, h, w = feats.shape
        feats = feats.flatten(2).transpose(1, 2)         # [B, H*W, C]
        # Set encoder_dim once (C)
        if self.encoder_dim is None:
            self.encoder_dim = feats.shape[-1]
        return feats




In [9]:
class Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        super(Attention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  # transform encoder output
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)  # transform decoder hidden
        self.full_att    = nn.Linear(attention_dim, 1)
        self.relu        = nn.ReLU()
        self.softmax     = nn.Softmax(dim=1)

    def forward(self, encoder_out, decoder_hidden):
        """
        encoder_out:    [batch_size, num_patches, encoder_dim]
        decoder_hidden: [batch_size, decoder_dim]
        """
        att1 = self.encoder_att(encoder_out)                  # [batch_size, num_patches, attention_dim]
        att2 = self.decoder_att(decoder_hidden).unsqueeze(1)  # [batch_size, 1, attention_dim]

        # sum -> relu -> full_att -> squeeze -> softmax
        att  = self.full_att(self.relu(att1 + att2)).squeeze(2)  # [batch_size, num_patches]
        alpha = self.softmax(att)
        # Weighted sum of the encoder_out
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  # [batch_size, encoder_dim]
        return attention_weighted_encoding, alpha

class DecoderRNN(nn.Module):
    def __init__(self, attention_dim, embed_dim, decoder_dim, vocab_size, encoder_dim=768, teacher_forcing_ratio=0.5):
        super(DecoderRNN, self).__init__()

        self.attention     = Attention(encoder_dim, decoder_dim, attention_dim)
        self.embedding     = nn.Embedding(vocab_size, embed_dim)
        self.dropout       = nn.Dropout(p=0.5)

        # [embed_dim + encoder_dim] -> decoder_dim
        self.lstm1 = nn.LSTMCell(embed_dim + encoder_dim, decoder_dim)
        # decoder_dim -> decoder_dim
        self.lstm2 = nn.LSTMCell(decoder_dim, decoder_dim)

        # For initializing the hidden states of both LSTM layers
        self.init_h1 = nn.Linear(encoder_dim, decoder_dim)
        self.init_c1 = nn.Linear(encoder_dim, decoder_dim)
        self.init_h2 = nn.Linear(encoder_dim, decoder_dim)
        self.init_c2 = nn.Linear(encoder_dim, decoder_dim)

        # Gating
        self.f_beta  = nn.Linear(decoder_dim, encoder_dim)
        self.sigmoid = nn.Sigmoid()

        # Final linear layer for output vocab
        self.fc = nn.Linear(decoder_dim, vocab_size)

        self.teacher_forcing_ratio = teacher_forcing_ratio

        self.init_weights()

    def init_weights(self):
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)

    def init_hidden_state(self, encoder_out):
        # encoder_out: [batch_size, num_patches, encoder_dim]
        mean_encoder_out = encoder_out.mean(dim=1)  # [batch_size, encoder_dim]
        h1 = self.init_h1(mean_encoder_out)         # [batch_size, decoder_dim]
        c1 = self.init_c1(mean_encoder_out)         # [batch_size, decoder_dim]
        h2 = self.init_h2(mean_encoder_out)         # [batch_size, decoder_dim]
        c2 = self.init_c2(mean_encoder_out)         # [batch_size, decoder_dim]
        return (h1, c1, h2, c2)

    def forward(self, encoder_out, encoded_captions, caption_lengths):
        """
        encoder_out:      [batch_size, num_patches, encoder_dim]
        encoded_captions: [batch_size, max_label_length]
        caption_lengths:  [batch_size, 1]
        """
        caption_lengths, sort_ind = caption_lengths.squeeze(1).sort(dim=0, descending=True)
        encoder_out      = encoder_out[sort_ind]
        encoded_captions = encoded_captions[sort_ind]

        embeddings = self.embedding(encoded_captions)

        # Initialize hidden states for both LSTM layers
        h1, c1, h2, c2 = self.init_hidden_state(encoder_out)

        decode_lengths    = (caption_lengths - 1).tolist()
        max_decode_length = max(decode_lengths)

        batch_size = encoder_out.size(0)
        vocab_size = self.fc.out_features

        predictions = torch.zeros(batch_size, max_decode_length, vocab_size, device=encoder_out.device)
        alphas      = torch.zeros(batch_size, max_decode_length, encoder_out.size(1), device=encoder_out.device)

        # We'll feed the first token from the input (<SOS>) or from the previous prediction
        prev_tokens = encoded_captions[:, 0].clone()

        for t in range(max_decode_length):
            batch_size_t = sum([l > t for l in decode_lengths])

            attention_weighted_encoding, alpha = self.attention(
                encoder_out[:batch_size_t],
                h1[:batch_size_t]  # use the first LSTM layer's hidden state for attention
            )

            # Apply gating
            gate = self.sigmoid(self.f_beta(h1[:batch_size_t]))
            attention_weighted_encoding = gate * attention_weighted_encoding

            # Teacher forcing?
            use_teacher_forcing = (torch.rand(1).item() < self.teacher_forcing_ratio)
            if use_teacher_forcing:
                current_input = embeddings[:batch_size_t, t, :]
            else:
                current_input = self.embedding(prev_tokens[:batch_size_t].detach())

            # first lstm layer
            h1_next, c1_next = self.lstm1(
                torch.cat([current_input, attention_weighted_encoding], dim=1),
                (h1[:batch_size_t], c1[:batch_size_t])
            )

            # second lstm layer
            h2_next, c2_next = self.lstm2(
                h1_next, (h2[:batch_size_t], c2[:batch_size_t])
            )

            # Use the second LSTM layer's output (h2_next) for final prediction
            preds = self.fc(self.dropout(h2_next))
            predictions[:batch_size_t, t, :] = preds
            alphas[:batch_size_t, t, :]      = alpha

            # Update prev_tokens with the best predicted token
            _, next_tokens = preds.max(dim=1)
            prev_tokens_ = prev_tokens.clone()
            prev_tokens_[:batch_size_t] = next_tokens.detach()
            prev_tokens = prev_tokens_

            # Update hidden states
            # For samples still in the batch, store the new h1, c1, h2, c2
            h1_new = torch.zeros_like(h1)
            c1_new = torch.zeros_like(c1)
            h2_new = torch.zeros_like(h2)
            c2_new = torch.zeros_like(c2)

            h1_new[:batch_size_t] = h1_next
            c1_new[:batch_size_t] = c1_next
            h2_new[:batch_size_t] = h2_next
            c2_new[:batch_size_t] = c2_next

            h1, c1, h2, c2 = h1_new, c1_new, h2_new, c2_new

        return predictions, encoded_captions, decode_lengths, alphas, sort_ind

In [10]:
class ImageCaptioningTrainer:
    def __init__(self, encoder, decoder, 
                 criterion, encoder_optimizer, decoder_optimizer, 
                 train_loader, val_loader, device, 
                 char_to_idx, idx_to_char, max_label_length,
                 model_name, csv_filename="training_results.csv"):
        self.encoder = encoder.to(device)
        self.decoder             = decoder.to(device)
        self.criterion           = criterion
        self.encoder_optimizer   = encoder_optimizer
        self.decoder_optimizer   = decoder_optimizer
        self.train_loader        = train_loader
        self.val_loader          = val_loader
        self.device              = device
        self.char_to_idx         = char_to_idx
        self.idx_to_char         = idx_to_char
        self.max_label_length    = max_label_length
        self.model_name = model_name
        self.csv_filename = csv_filename

        self.train_losses = []
        self.val_losses   = []

        self.train_cers   = []
        self.val_cers     = []

    def fit(self, num_epochs):
        start_time = time.time()

        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch + 1}/{num_epochs}")
            train_loss, train_cer = self.train_one_epoch()
            val_loss,   val_cer   = self.validate_one_epoch(top_n=5)

            print(f"[{epoch+1}/{num_epochs}] "
                  f"Train Loss: {train_loss:.4f}, Train CER: {train_cer:.4f} | "
                  f"Val Loss: {val_loss:.4f}, Val CER: {val_cer:.4f}")

            # Store epoch results
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_cers.append(train_cer)
            self.val_cers.append(val_cer)
            
        
        # Calculate total training time
        end_time = time.time() 
        total_time = end_time - start_time
        hours = int(total_time // 3600)
        minutes = int((total_time % 3600) // 60)

        print(f"\nTraining completed in {hours}h {minutes}m.")

        num_epochs = len(self.train_losses)
        epoch_cols = [f"epoch{i+1}" for i in range(num_epochs)]

        # Create the new data block to insert
        new_rows = pd.DataFrame([
            [self.model_name, "training loss"] + self.train_losses,
            [self.model_name, "validation loss"] + self.val_losses,
            [self.model_name, "training cer"] + self.train_cers,
            [self.model_name, "validation cer"] + self.val_cers
        ], columns=["model_name", "mode"] + epoch_cols)
        
        # Check if CSV already exists
        if os.path.exists(self.csv_filename):
            df_existing = pd.read_csv(self.csv_filename)
            df_existing = df_existing[df_existing["model_name"] != self.model_name]
            df_updated = pd.concat([df_existing, new_rows], ignore_index=True)
        else:
            df_updated = new_rows
        
        # Save the updated CSV
        df_updated.to_csv(self.csv_filename, index=False)
        print(f"\nResults have been written to: {self.csv_filename}")


        # Save model weights
        torch.save(self.encoder.state_dict(), f"encoder_{self.model_name}.pth")
        torch.save(self.decoder.state_dict(), f"decoder_{self.model_name}.pth")
        print(f"Encoder and decoder models saved: encoder_{self.model_name}.pth, decoder_{self.model_name}.pth")
        

    def train_one_epoch(self):
        self.encoder.train()
        self.decoder.train()
        running_loss           = 0.0
        total_edit_distance    = 0
        total_ref_length       = 0

        for batch_idx, (images, labels, label_lengths) in enumerate(self.train_loader):
            images        = images.to(self.device, non_blocking=True)
            labels        = labels.to(self.device, non_blocking=True)
            label_lengths = label_lengths.to(self.device, non_blocking=True)

            self.encoder_optimizer.zero_grad()
            self.decoder_optimizer.zero_grad()

            encoder_out   = self.encoder(images)
            caption_lengths = torch.tensor(
                [self.max_label_length] * labels.size(0)
            ).unsqueeze(1).to(self.device)

            outputs, encoded_captions, decode_lengths, alphas, sort_ind = self.decoder(
                encoder_out, labels, caption_lengths
            )

            # Targets = encoded captions without the <SOS>
            targets = encoded_captions[:, 1:]

            # Flatten for loss
            outputs_flat = outputs.view(-1, self.decoder.fc.out_features)
            targets_flat = targets.contiguous().view(-1)

            loss = self.criterion(outputs_flat, targets_flat)
            loss.backward()

            self.decoder_optimizer.step()
            self.encoder_optimizer.step()

            running_loss += loss.item()

            # Compute CER for the batch (global style)
            batch_size = labels.size(0)
            _, preds_flat = torch.max(outputs_flat, dim=1)
            preds_seq = preds_flat.view(batch_size, -1)

            for i in range(batch_size):
                pred_indices   = preds_seq[i].detach().cpu().numpy()
                target_indices = targets[i].detach().cpu().numpy()

                mask          = (target_indices != self.char_to_idx['<PAD>'])
                pred_indices  = pred_indices[mask]
                target_indices= target_indices[mask]

                pred_chars    = [self.idx_to_char.get(idx, '') for idx in pred_indices]
                target_chars  = [self.idx_to_char.get(idx, '') for idx in target_indices]
                pred_str      = ''.join(pred_chars)
                target_str    = ''.join(target_chars)

                edit_dist           = editdistance.eval(pred_str, target_str)
                total_edit_distance += edit_dist
                total_ref_length    += len(target_str)

            # if (batch_idx + 1) % 50 == 0:
            #     print(f'Batch {batch_idx + 1}/{len(self.train_loader)} - Loss: {loss.item():.4f}')

        avg_loss = running_loss / len(self.train_loader)
        avg_cer  = total_edit_distance / total_ref_length if total_ref_length > 0 else 0.0
        return avg_loss, avg_cer

    def validate_one_epoch(self, top_n=5):
        self.encoder.eval()
        self.decoder.eval()
        running_loss         = 0.0
        total_edit_distance  = 0
        total_ref_length     = 0

        # each sample’s CER
        sample_cer_info = []

        with torch.no_grad():
            for batch_idx, (images, labels, label_lengths) in enumerate(self.val_loader):
                images        = images.to(self.device, non_blocking=True)
                labels        = labels.to(self.device, non_blocking=True)
                label_lengths = label_lengths.to(self.device, non_blocking=True)

                encoder_out = self.encoder(images)
                caption_lengths = torch.tensor(
                    [self.max_label_length] * labels.size(0)
                ).unsqueeze(1).to(self.device)

                outputs, encoded_captions, decode_lengths, alphas, sort_ind = self.decoder(
                    encoder_out, labels, caption_lengths
                )
                targets = encoded_captions[:, 1:]

                outputs_flat = outputs.view(-1, self.decoder.fc.out_features)
                targets_flat = targets.contiguous().view(-1)

                loss = self.criterion(outputs_flat, targets_flat)
                running_loss += loss.item()

                batch_size = labels.size(0)
                _, preds_flat = torch.max(outputs_flat, dim=1)
                preds_seq = preds_flat.view(batch_size, -1)

                for i in range(batch_size):
                    pred_indices   = preds_seq[i].detach().cpu().numpy()
                    target_indices = targets[i].detach().cpu().numpy()

                    mask           = (target_indices != self.char_to_idx['<PAD>'])
                    pred_indices   = pred_indices[mask]
                    target_indices = target_indices[mask]

                    pred_chars   = [self.idx_to_char.get(idx, '') for idx in pred_indices]
                    target_chars = [self.idx_to_char.get(idx, '') for idx in target_indices]
                    pred_str     = ''.join(pred_chars)
                    target_str   = ''.join(target_chars)

                    edit_dist = editdistance.eval(pred_str, target_str)
                    ref_len   = len(target_str)
                    cer       = edit_dist / ref_len if ref_len > 0 else 0
    
                    total_edit_distance += edit_dist
                    total_ref_length    += ref_len
    
                    # Store sample info
                    # sample_cer_info.append({
                    #     "pred": pred_str,
                    #     "gt": target_str,
                    #     "cer": cer
                    # })

                    # Print a few samples from the 1st batch
                    # if batch_idx == 0 and i < 3:
                    #     print(f"Sample {i + 1}:")
                    #     print(f"Predicted: {pred_str}")
                    #     print(f"Target   : {target_str}\n")

        avg_loss = running_loss / len(self.val_loader)
        avg_cer  = total_edit_distance / total_ref_length if total_ref_length > 0 else 0.0

        # Sort by CER descending
        sample_cer_info.sort(key=lambda x: x["cer"], reverse=True)
        # Take top_n
        worst_samples = sample_cer_info[:top_n]
    
        # print(f"\n=== Top {top_n} Worst Samples by CER ===")
        # for idx, sample in enumerate(worst_samples):
        #     print(f"[{idx+1}] CER: {sample['cer']:.3f}")
        #     print(f"   Predicted: {sample['pred']}")
        #     print(f"   Ground Truth: {sample['gt']}\n")
       
        return avg_loss, avg_cer

In [11]:
cnn_encoder = ResNet18Encoder(pretrained=True)
cnn_decoder = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=cnn_encoder.encoder_dim,  # 512 for ResNet18
    teacher_forcing_ratio=0.5
)

cnn_encoder.to(device)
cnn_decoder.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=char_to_idx['<PAD>'])
encoder_optimizer = optim.Adam(cnn_encoder.parameters(), lr=1e-4)
decoder_optimizer = optim.Adam(cnn_decoder.parameters(), lr=4e-4)

trainer = ImageCaptioningTrainer(
    encoder=cnn_encoder,
    decoder=cnn_decoder,
    criterion=criterion,
    encoder_optimizer=encoder_optimizer,
    decoder_optimizer=decoder_optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_label_length=max_label_length,
    model_name="resnet18_encoder"
)

num_epochs = 2
trainer.fit(num_epochs)

/apps/miniforge3/24.3.0-0/miniforge3/envs/dsks_2024.06/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/apps/miniforge3/24.3.0-0/miniforge3/envs/dsks_2024.06/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/2
[1/2] Train Loss: 1.9205, Train CER: 0.6453 | Val Loss: 1.1061, Val CER: 0.2955

Epoch 2/2
[2/2] Train Loss: 1.1015, Train CER: 0.3105 | Val Loss: 0.5574, Val CER: 0.1482

Training completed in 0h 1m.

Results have been written to: training_results.csv
Encoder and decoder models saved: encoder_resnet18_encoder.pth, decoder_resnet18_encoder.pth


In [12]:
# Manually delete references to free GPU memory
del cnn_encoder
del cnn_decoder
del trainer

# Empty the PyTorch CUDA cache
torch.cuda.empty_cache()
print("Memory cleared")

Memory cleared


In [13]:
cnn_encoder_50 = ResNet50Encoder(pretrained=True)
cnn_decoder_50 = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=cnn_encoder_50.encoder_dim,  # 2048 for ResNet50
    teacher_forcing_ratio=0.5
)

cnn_encoder_50 = cnn_encoder_50.to(device)
cnn_decoder_50 = cnn_decoder_50.to(device)

trainer = ImageCaptioningTrainer(
    encoder=cnn_encoder_50,
    decoder=cnn_decoder_50,
    criterion=criterion,
    encoder_optimizer=encoder_optimizer,
    decoder_optimizer=decoder_optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_label_length=max_label_length,
    model_name="resnet50_encoder"
)
num_epochs = 2
trainer.fit(num_epochs)

/apps/miniforge3/24.3.0-0/miniforge3/envs/dsks_2024.06/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/apps/miniforge3/24.3.0-0/miniforge3/envs/dsks_2024.06/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/yhuang1/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 98.1MB/s]



Epoch 1/2
[1/2] Train Loss: 3.6586, Train CER: 0.9567 | Val Loss: 3.6516, Val CER: 0.9770

Epoch 2/2
[2/2] Train Loss: 3.6586, Train CER: 0.9579 | Val Loss: 3.6511, Val CER: 0.9713

Training completed in 0h 1m.

Results have been written to: training_results.csv
Encoder and decoder models saved: encoder_resnet50_encoder.pth, decoder_resnet50_encoder.pth


In [14]:
# Manually delete references to free GPU memory
del cnn_encoder_50
del cnn_decoder_50
del trainer

# Empty the PyTorch CUDA cache
torch.cuda.empty_cache()
print("Memory cleared")

Memory cleared


In [15]:
encoder_vit_base = ViTEncoder(model_name="vit_base_patch16_224", pretrained=True)
decoder_vit_base = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=encoder_vit_base.encoder_dim,  
    teacher_forcing_ratio=0.5
).to(device)

encoder_vit_base = encoder_vit_base.to(device)
decoder_vit_base = decoder_vit_base.to(device)


criterion = nn.CrossEntropyLoss(ignore_index=char_to_idx['<PAD>'])
encoder_optimizer_base = optim.Adam(encoder_vit_base.parameters(), lr=1e-4)
decoder_optimizer_base = optim.Adam(decoder_vit_base.parameters(), lr=4e-4)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [16]:
trainer_base = ImageCaptioningTrainer(
    encoder=encoder_vit_base,
    decoder=decoder_vit_base,
    criterion=criterion,                   
    encoder_optimizer=encoder_optimizer_base,
    decoder_optimizer=decoder_optimizer_base,
    train_loader=train_loader,             
    val_loader=val_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_label_length=max_label_length,
    model_name="vit_base_patch16_224"     
)

num_epochs = 2
trainer_base.fit(num_epochs)


Epoch 1/2
[1/2] Train Loss: 1.9833, Train CER: 0.6515 | Val Loss: 1.1742, Val CER: 0.3308

Epoch 2/2
[2/2] Train Loss: 1.1646, Train CER: 0.3502 | Val Loss: 0.6110, Val CER: 0.1587

Training completed in 0h 4m.

Results have been written to: training_results.csv
Encoder and decoder models saved: encoder_vit_base_patch16_224.pth, decoder_vit_base_patch16_224.pth


In [17]:
# Manually delete references to free GPU memory
del encoder_vit_base
del decoder_vit_base
del trainer_base

# Empty the PyTorch CUDA cache
torch.cuda.empty_cache()
print("Memory cleared")

Memory cleared


In [18]:
encoder_vit_large = ViTEncoder(model_name="vit_large_patch16_224", pretrained=True)
decoder_vit_large = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=encoder_vit_large.encoder_dim,
    teacher_forcing_ratio=0.5
)

encoder_vit_large = encoder_vit_large.to(device)
decoder_vit_large = decoder_vit_large.to(device)

encoder_optimizer_large = optim.Adam(encoder_vit_large.parameters(), lr=1e-4)
decoder_optimizer_large = optim.Adam(decoder_vit_large.parameters(), lr=4e-4)

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [19]:
trainer_large = ImageCaptioningTrainer(
    encoder=encoder_vit_large,
    decoder=decoder_vit_large,
    criterion=criterion,                    
    encoder_optimizer=encoder_optimizer_large,
    decoder_optimizer=decoder_optimizer_large,
    train_loader=train_loader,              
    val_loader=val_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_label_length=max_label_length,
    model_name="vit_large_patch16_224"     
)

num_epochs = 2
trainer_large.fit(num_epochs)


Epoch 1/2
[1/2] Train Loss: 1.9298, Train CER: 0.6421 | Val Loss: 1.0167, Val CER: 0.2943

Epoch 2/2
[2/2] Train Loss: 1.0843, Train CER: 0.3192 | Val Loss: 0.4924, Val CER: 0.1296

Training completed in 0h 12m.

Results have been written to: training_results.csv
Encoder and decoder models saved: encoder_vit_large_patch16_224.pth, decoder_vit_large_patch16_224.pth


In [20]:
# Manually delete references to free GPU memory
del encoder_vit_large
del decoder_vit_large
del trainer_large

# Empty the PyTorch CUDA cache
torch.cuda.empty_cache()
print("Memory cleared")

Memory cleared


In [21]:
swin_encoder = SwinEncoder(model_name="swin_small_patch4_window7_224", pretrained=True).to(device)

# Dummy forward to set encoder_dim
with torch.no_grad():
    dummy = torch.randn(1, 3, 224, 224, device=device)
    _ = swin_encoder(dummy)

# Now build decoder with the correct encoder_dim
swin_decoder = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=swin_encoder.encoder_dim,  
    teacher_forcing_ratio=0.5
).to(device)


# Loss + optimizers
criterion = nn.CrossEntropyLoss(ignore_index=char_to_idx['<PAD>'])
encoder_optimizer = optim.Adam(swin_encoder.parameters(), lr=1e-4)
decoder_optimizer = optim.Adam(swin_decoder.parameters(), lr=4e-4)

# Trainer
trainer = ImageCaptioningTrainer(
    encoder=swin_encoder,
    decoder=swin_decoder,
    criterion=criterion,
    encoder_optimizer=encoder_optimizer,
    decoder_optimizer=decoder_optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_label_length=max_label_length,
    model_name="swin_small_encoder"
)

# Train for 2 epochs
num_epochs = 2
trainer.fit(num_epochs)

model.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]


Epoch 1/2
[1/2] Train Loss: 2.2736, Train CER: 0.8234 | Val Loss: 1.8169, Val CER: 0.4870

Epoch 2/2
[2/2] Train Loss: 2.0163, Train CER: 0.6097 | Val Loss: 1.7290, Val CER: 0.4487

Training completed in 0h 4m.

Results have been written to: training_results.csv
Encoder and decoder models saved: encoder_swin_small_encoder.pth, decoder_swin_small_encoder.pth


In [22]:
class TrOCREncoderDecoder(nn.Module):
    def __init__(self, model_name="microsoft/trocr-small-handwritten"):
        super().__init__()
        self.processor = TrOCRProcessor.from_pretrained(model_name)
        self.model     = VisionEncoderDecoderModel.from_pretrained(model_name)
        self.encoder_dim = self.model.config.encoder.hidden_size
        self.decoder_dim = self.model.config.decoder.hidden_size

        # align special tokens
        self.model.config.decoder_start_token_id = self.processor.tokenizer.cls_token_id
        self.model.config.pad_token_id           = self.processor.tokenizer.pad_token_id

    def forward(self, images, captions=None):
        """
        images: list of PIL.Image or tensor [B,3,H,W]
        captions: tensor [B, max_len] (token ids)
        """
        pixel_values = self.processor(images=images, return_tensors="pt").pixel_values.to(self.model.device)
        outputs = self.model(pixel_values=pixel_values, labels=captions.to(self.model.device) if captions is not None else None)
        return outputs

# Instantiate
trocr = TrOCREncoderDecoder().to(device)

# Replace criterion with built‑in cross‑entropy in the model (ignore index = pad)
criterion = lambda outputs, labels: outputs.loss

encoder_optimizer = optim.Adam(trocr.model.encoder.parameters(), lr=3e-5)
decoder_optimizer = optim.Adam(trocr.model.decoder.parameters(), lr=5e-5)

# Re‑use your ImageCaptioningTrainer — swap encoder+decoder for trocr.model
trainer = ImageCaptioningTrainer(
    encoder=trocr.model.encoder,
    decoder=trocr.model.decoder,
    criterion=criterion,
    encoder_optimizer=encoder_optimizer,
    decoder_optimizer=decoder_optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    char_to_idx=trocr.processor.tokenizer.get_vocab(),
    idx_to_char={i:t for t,i in trocr.processor.tokenizer.get_vocab().items()},
    max_label_length=max_label_length,
    model_name="trocr_small"
)

# Train for 20 epochs
trainer.fit(num_epochs=2)


/home/yhuang1/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

2025-03-25 19:29:37.091483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-25 19:29:38.974308: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 19:29:40.659882: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 19:29:44.053126: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [23]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

def load_trocr_model(model_name="microsoft/trocr-base-handwritten", device=None):
    """
    Safely load TrOCR model with memory-efficient strategies
    
    Args:
        model_name (str): Hugging Face model identifier
        device (str, optional): Computation device ('cuda', 'cpu')
    
    Returns:
        tuple: (processor, model)
    """
    # Determine optimal device
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Check available memory
    if device == "cuda":
        total_memory = torch.cuda.get_device_properties(0).total_memory
        print(f"Total GPU Memory: {total_memory / (1024**3):.2f} GB")
    
    # Load processor and model
    processor = TrOCRProcessor.from_pretrained(model_name)
    model = VisionEncoderDecoderModel.from_pretrained(model_name)
    
    # Move model to appropriate device
    model.to(device)
    
    # Optional: Use half precision to reduce memory
    if device == "cuda":
        model = model.half()
    
    return processor, model

def memory_check():
    """
    Perform comprehensive memory diagnostics
    """
    print("Memory Diagnostics:")
    print(f"PyTorch Version: {torch.__version__}")
    print(f"CUDA Available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Current Device: {torch.cuda.current_device()}")
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"GPU Memory Cached: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")

# Example usage
memory_check()
processor, model = load_trocr_model()

Memory Diagnostics:
PyTorch Version: 2.0.1
CUDA Available: True
CUDA Device Name: NVIDIA A40
CUDA Current Device: 0
GPU Memory Allocated: 7116.26 MB
GPU Memory Cached: 16094.00 MB
Total GPU Memory: 44.35 GB


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight', 'decoder.output_projection.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
test_ground_truth_path = os.path.join(base_dir, 'balinese_transliteration_test.txt')
test_images_dir        = os.path.join(base_dir, 'balinese_word_test')

test_filenames = []
test_labels    = []

with open(test_ground_truth_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
            parts = line.split(';')
            if len(parts) == 2:
                filename, label = parts
                label = label.lower()
                test_filenames.append(filename)
                test_labels.append(label)
            else:
                print(f"Skipping malformed line: {line}")

test_data = pd.DataFrame({
    'filename': test_filenames,
    'label': test_labels
})

# Check for unknown chars in test set
test_chars = set(''.join(test_data['label']))
unknown_chars = test_chars - set(char_to_idx.keys())
print(f"Unknown characters in test labels: {unknown_chars}")

# Encode test labels
max_label_length_test = max(len(lbl) for lbl in test_data['label']) + 2
def encode_label_test(label, char_to_idx, max_length):
    encoded = (
        [char_to_idx['<SOS>']] +
        [char_to_idx.get(ch, char_to_idx['<UNK>']) for ch in label] +
        [char_to_idx['<EOS>']]
    )
    if len(encoded) > max_length:
        encoded = encoded[:max_length]
    else:
        encoded += [char_to_idx['<PAD>']] * (max_length - len(encoded))
    return encoded

test_data['encoded_label'] = test_data['label'].apply(lambda x: encode_label_test(x, char_to_idx, max_label_length_test))
test_data['label_length']  = test_data['label'].apply(len)

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

test_dataset = BalineseDataset(test_data, test_images_dir, transform=test_transform)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
def inference(encoder, decoder, data_loader, device, char_to_idx, idx_to_char, max_seq_length, test_data):
    encoder.eval()
    decoder.eval()

    eos_idx = char_to_idx['<EOS>']
    results = []

    with torch.no_grad():
        for batch_idx, (images, labels, label_lengths) in enumerate(data_loader):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            batch_size  = images.size(0)
            encoder_out = encoder(images)  # [B, num_patches, encoder_dim]

            # Init LSTM state
            h1, c1, h2, c2 = decoder.init_hidden_state(encoder_out)

            # Start tokens (all <SOS>)
            inputs = torch.full(
                (batch_size,),
                fill_value=char_to_idx['<SOS>'],
                dtype=torch.long,
                device=device
            )

            all_preds = []

            for _ in range(max_seq_length):
                # 1) Embedding
                embeddings = decoder.embedding(inputs)

                # 2) Attention
                attention_weighted_encoding, _ = decoder.attention(encoder_out, h1)

                # 3) Gating
                gate = decoder.sigmoid(decoder.f_beta(h1))
                attention_weighted_encoding = gate * attention_weighted_encoding

                # 4) Pass through LSTM layers
                h1, c1 = decoder.lstm1(
                    torch.cat([embeddings, attention_weighted_encoding], dim=1),
                    (h1, c1)
                )
                h2, c2 = decoder.lstm2(h1, (h2, c2))

                # 5) Get predicted token
                preds = decoder.fc(decoder.dropout(h2))  # [batch_size, vocab_size]
                _, preds_idx = preds.max(dim=1)

                # Feed next token
                all_preds.append(preds_idx.cpu().numpy())
                inputs = preds_idx

            # Reformat predictions to [batch_size, max_seq_length]
            all_preds = np.array(all_preds).T

            for i in range(batch_size):
                pred_indices = all_preds[i]

                # Stop at <EOS> if present
                if eos_idx in pred_indices:
                    first_eos = np.where(pred_indices == eos_idx)[0][0]
                    pred_indices = pred_indices[:first_eos]

                # Convert token indices -> string
                pred_chars = [idx_to_char.get(idx, '') for idx in pred_indices]
                pred_str   = ''.join(pred_chars)

                # Process ground truth
                label_indices = labels[i].cpu().numpy()
                # remove <SOS>
                label_indices = label_indices[1:]

                if eos_idx in label_indices:
                    eos_pos = np.where(label_indices == eos_idx)[0][0]
                    label_indices = label_indices[:eos_pos]
                else:
                    # remove <PAD> if present
                    label_indices = label_indices[label_indices != char_to_idx['<PAD>']]

                label_chars = [idx_to_char.get(idx, '') for idx in label_indices]
                label_str   = ''.join(label_chars)

                global_idx    = batch_idx * batch_size + i
                image_filename= test_data.iloc[global_idx]['filename']

                results.append({
                    'image_filename': image_filename,
                    'predicted_caption': pred_str,
                    'ground_truth_caption': label_str
                })

    return results

In [ ]:
def calculate_global_cer(results):
    total_ed   = 0
    total_refs = 0
    for r in results:
        ref = r['ground_truth_caption']
        hyp = r['predicted_caption']
        dist = editdistance.eval(ref, hyp)
        total_ed   += dist
        total_refs += len(ref)
    if total_refs == 0:
        return 0.0
    return total_ed / total_refs

In [ ]:
def print_top_worst_samples(results, n=5):
    # Calculate CER for each sample
    results_with_cer = []
    for r in results:
        ref = r['ground_truth_caption']
        hyp = r['predicted_caption']
        dist = editdistance.eval(ref, hyp)
        length = len(ref)
        cer = dist / length if length > 0 else 0
        # Copy the record and add cer
        new_r = r.copy()
        new_r['cer'] = cer
        results_with_cer.append(new_r)

    # Sort by CER (descending) and take the top N
    results_with_cer.sort(key=lambda x: x['cer'], reverse=True)
    worst_samples = results_with_cer[:n]

    print(f"\n=== Top {n} Worst Samples by CER ===")
    for i, sample in enumerate(worst_samples, start=1):
        print(f"{i}) Image: {sample['image_filename']}")
        print(f"   CER: {sample['cer']:.4f}")
        print(f"   Predicted       : {sample['predicted_caption']}")
        print(f"   Ground Truth    : {sample['ground_truth_caption']}")
        print()

In [ ]:
cnn_encoder = ResNet18Encoder(pretrained=True)
cnn_decoder = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=cnn_encoder.encoder_dim,  # 512 for ResNet18
    teacher_forcing_ratio=0.5
)

encoder_ckpt = "encoder_resnet18_encoder.pth"
decoder_ckpt = "decoder_resnet18_encoder.pth"

cnn_encoder.load_state_dict(torch.load(encoder_ckpt, map_location=device))
cnn_decoder.load_state_dict(torch.load(decoder_ckpt, map_location=device))

cnn_encoder = cnn_encoder.to(device)
cnn_decoder = cnn_decoder.to(device)

cnn_encoder.eval()
cnn_decoder.eval()

test_results = inference(
    encoder=cnn_encoder,
    decoder=cnn_decoder,
    data_loader=test_loader,             
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_seq_length=max_label_length_test, 
    test_data=test_data                   
)

global_cer = calculate_global_cer(test_results)
print(f"Global CER on test set: {global_cer:.4f}")

print_top_worst_samples(test_results, n=5)

In [ ]:
cnn_encoder_50 = ResNet50Encoder(pretrained=True)
cnn_decoder_50 = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=cnn_encoder_50.encoder_dim,  # 2048 for ResNet50
    teacher_forcing_ratio=0.5
)

encoder_ckpt = "encoder_resnet50_encoder.pth"
decoder_ckpt = "decoder_resnet50_encoder.pth"

cnn_encoder_50.load_state_dict(torch.load(encoder_ckpt, map_location=device))
cnn_decoder_50.load_state_dict(torch.load(decoder_ckpt, map_location=device))

# Move to device
cnn_encoder_50 = cnn_encoder_50.to(device)
cnn_decoder_50 = cnn_decoder_50.to(device)

cnn_encoder_50.eval()
cnn_decoder_50.eval()

test_results = inference(
    encoder=cnn_encoder_50,
    decoder=cnn_decoder_50,
    data_loader=test_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_seq_length=max_label_length_test,
    test_data=test_data
)

global_cer = calculate_global_cer(test_results)
print(f"Global CER on test set: {global_cer:.4f}")

print_top_worst_samples(test_results, n=5)


In [ ]:
encoder = ViTEncoder(model_name="vit_base_patch16_224", pretrained=True).to(device)
decoder = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=encoder.encoder_dim, 
    teacher_forcing_ratio=0.5
).to(device)

encoder.load_state_dict(torch.load("encoder_vit_base_patch16_224.pth", map_location=device))
decoder.load_state_dict(torch.load("decoder_vit_base_patch16_224.pth", map_location=device))

test_results = inference(
    encoder=encoder,
    decoder=decoder,
    data_loader=test_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_seq_length=max_label_length_test,
    test_data=test_data
)

global_cer = calculate_global_cer(test_results)
print(f"Global CER on Test Set (ViT-Base) : {global_cer:.4f}")

print_top_worst_samples(test_results, n=5)

In [ ]:
encoder_large = ViTEncoder(model_name="vit_large_patch16_224", pretrained=True).to(device)
decoder_large = DecoderRNN(
    attention_dim=256,
    embed_dim=256,
    decoder_dim=512,
    vocab_size=vocab_size,
    encoder_dim=encoder_large.encoder_dim,
    teacher_forcing_ratio=0.5
).to(device)

encoder_large.load_state_dict(torch.load("encoder_vit_large_patch16_224.pth", map_location=device))
decoder_large.load_state_dict(torch.load("decoder_vit_large_patch16_224.pth", map_location=device))

test_results_large = inference(
    encoder=encoder_large,
    decoder=decoder_large,
    data_loader=test_loader,
    device=device,
    char_to_idx=char_to_idx,
    idx_to_char=idx_to_char,
    max_seq_length=max_label_length_test,
    test_data=test_data
)

global_cer_large = calculate_global_cer(test_results_large)
print(f"\nGlobal CER on Test Set (ViT-Large): {global_cer_large:.4f}")

print_top_worst_samples(test_results_large, n=5)
